### clear general table

In [165]:
Data = 'data/SQL/data.sqlite3'
import sqlite3

def Delete_data_table(filename, name_table):
    conn = sqlite3.connect(filename)
    cursor = conn.cursor()
    cursor.execute("DELETE FROM " + name_table + ";")
    conn.commit() 
    tables = cursor.fetchall()
    conn.close()
    return [table[0] for table in tables]

Delete_data_table(Data,"'General'")

[]

### File data.general avec 'info_student' & 'Sondage_LV2'

In [166]:

import sqlite3
import pandas as pd
import numpy as np
import json
import os


depot_info_folder = './data/depot_info'
db_path = './data/SQL/data.sqlite3' 


def file_data_general(depot_info_folder, db_path):
    conn = sqlite3.connect(db_path)
    desired_table_name = 'General'
    for file in [f for f in os.listdir(depot_info_folder)]:
        if os.path.splitext(os.path.basename(file))[0] == 'Info_student' :
            db_column_mapping = {
                'Nom': 'NAME',
                'Prénom': 'SURNAME',
                'mail': 'EMAIL',
                'Class': 'CLASS'}

            if file.endswith('.csv'):
                csv_file_path = os.path.join(depot_info_folder, file)
                df = pd.read_csv(csv_file_path, encoding='utf-8-sig')                
                df.rename(columns=db_column_mapping, inplace=True)
                df = df[list(db_column_mapping.values())]
                df.to_sql(desired_table_name, conn, if_exists='append', index=False)

            elif file.endswith('.json'):
                json_file_path = os.path.join(depot_info_folder, file)
                with open(json_file_path, 'r', encoding='utf-8-sig') as json_file:
                    data = json.load(json_file)
                    transformed_data = []
                    for item in data:
                        transformed_item = {db_column: item.get(json_field) for json_field, db_column in db_column_mapping.items()}
                        transformed_data.append(transformed_item)
                    df = pd.DataFrame(transformed_data)
                    df.to_sql(desired_table_name, conn, if_exists='append', index=False)

            elif file.endswith('.xlsx'):
                xlsx_file_path = os.path.join(depot_info_folder, file)
                df = pd.read_excel(xlsx_file_path)
                df.rename(columns=db_column_mapping, inplace=True)
                df = df[list(db_column_mapping.values())]
                df.to_sql(desired_table_name, conn, if_exists='append', index=False)

            else:
                print(f"Format de fichier non pris en charge: {file}")

        elif os.path.splitext(os.path.basename(file))[0] == 'Sondage_LV2' :
            db_column_mapping = {
                'Nom': 'NAME',
                'Prénom': 'SURNAME',
                'mail': 'EMAIL',
                'Langues' : 'LV2'
            }
            if file.endswith('.csv'):
                csv_file_path = os.path.join(depot_info_folder, file)
                with open(csv_file_path, 'r', encoding='utf-8-sig') as csvfile:
                    csv_reader = pd.read_csv(csv_file_path, encoding='utf-8-sig')
                    for ligne in csv_reader:
                        ligne_filtered = {key: value for key, value in ligne.items() if key in db_column_mapping.keys()}
                        cursor_destination = conn.cursor()
                        cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;", (ligne_filtered["Nom"], ligne_filtered["Prénom"], ligne_filtered["mail"]))
                        count = cursor_destination.fetchone()[0]
                        if count == 0:
                            print(f"{ligne_filtered['Nom']} {ligne_filtered['Prénom']} doesn't exist")
                        else:
                            cursor_destination.execute("UPDATE General SET LV2=?, STATUS=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["Langues"],"PRESENT", ligne_filtered["Nom"], ligne_filtered["Prénom"], ligne_filtered["mail"]))
                        conn.commit()

            elif file.endswith('.json'):
                json_file_path = os.path.join(depot_info_folder, file)
                with open(json_file_path, 'r', encoding='utf-8-sig') as json_file:
                    data = json.load(json_file)
                    for ligne in data:
                        ligne_filtered = {key: value for key, value in ligne.items() if key in db_column_mapping.keys()}
                        cursor_destination = conn.cursor()
                        cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;", (ligne_filtered["Nom"], ligne_filtered["Prénom"], ligne_filtered["mail"]))
                        count = cursor_destination.fetchone()[0]
                        if count == 0:
                            print(f"{ligne_filtered['Nom']} {ligne_filtered['Prénom']} doesn't exist")
                        else:
                            cursor_destination.execute("UPDATE General SET LV2=?, STATUS=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["Langues"],"PRESENT", ligne_filtered["Nom"], ligne_filtered["Prénom"], ligne_filtered["mail"]))
                        conn.commit()

            elif file.endswith('.xlsx'):
                xlsx_file_path = os.path.join(depot_info_folder, file)
                df = pd.read_excel(xlsx_file_path)
                df = df[list(db_column_mapping.keys())]
                cursor_destination = conn.cursor()
                for index, ligne in df.iterrows():
                    cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;", (ligne["Nom"], ligne["Prénom"], ligne["mail"]))
                    count = cursor_destination.fetchone()[0]
                    if count == 0:
                        print(f"{ligne['Nom']} {ligne['Prénom']} doesn't exist")
                    else:
                        cursor_destination.execute("UPDATE General SET LV2=?, STATUS=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["Langues"],"PRESENT", ligne["Nom"], ligne["Prénom"], ligne["mail"]))
                conn.commit()

            else:
                print(f"Format de fichier non pris en charge: {file}")

    conn.close()
    return

file_data_general(depot_info_folder,db_path)

### File data.general avec $note_langue

comment savoir TT pour etudiant debutant dans un langue avec sondage anglais ???

In [167]:
import sqlite3
import pandas as pd
import numpy as np
import json
import os

def load_survey(depot_note_folder,db_path ):
    conn = sqlite3.connect(db_path)

    for file in [f for f in os.listdir(depot_note_folder)]:

        if "Anglais" not in os.path.splitext(os.path.basename(file))[0] :
            db_column_mapping = {
                    'Nom': 'NAME',
                    'Prénom': 'SURNAME',
                    'Mail': 'EMAIL',
                    'Note/10' : 'GRADE_LV2'
                    }
            
            if file.endswith('.csv'):
                csv_file_path = os.path.join(depot_note_folder, file)
                with open(csv_file_path, 'r', encoding='utf-8-sig') as csvfile:
                    csv_reader = pd.read_csv(csv_file_path, encoding='utf-8-sig')
                    csv_reader['Note/10'] = csv_reader['Note/10'].replace('', np.nan)
                    csv_reader['Note/10'] = csv_reader['Note/10'].astype(float)
                    for ligne in csv_reader:
                        ligne_filtered = {key: value for key, value in ligne.items() if key in db_column_mapping.keys()}
                        cursor_destination = conn.cursor()
                        cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;", (ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))                    
                        count = cursor_destination.fetchone()[0]
                        if count == 0:
                            print(f"{ligne_filtered['NAME']} {ligne_filtered['SURNAME']} doesn't exist")
                        else:
                            cursor_destination.execute("UPDATE General SET GRADE_LV2=?, EXTRA_TIME=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["GRADE_LV2"], extra_time, ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                        conn.commit()
                            
            elif file.endswith('.json'):
                json_file_path = os.path.join(depot_note_folder, file)
                with open(json_file_path, 'r', encoding='utf-8-sig') as jsonfile:
                    data = json.load(jsonfile)
                    df = pd.DataFrame(data)
                    df['Note/10'] = df['Note/10'].replace('', np.nan).astype(float)
                    for index, row in df.iterrows():
                        ligne_filtered = {db_column_mapping[key]: row[key] for key in row.keys() if key in db_column_mapping.keys()}
                        cursor_destination = conn.cursor()
                        cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;",(ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                        count = cursor_destination.fetchone()[0]
                        if count == 0:
                            print(f"{ligne_filtered['NAME']} {ligne_filtered['SURNAME']} doesn't exist")
                        else:
                            cursor_destination.execute("UPDATE General SET GRADE_LV2=?, EXTRA_TIME=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["GRADE_LV2"], extra_time, ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                        conn.commit()

            elif file.endswith('.xlsx'):
                xlsx_file_path = os.path.join(depot_note_folder, file)
                df = pd.read_excel(xlsx_file_path)
                df['Note/10'] = df['Note/10'].replace('', np.nan).astype(float)
                for index, row in df.iterrows():
                    ligne_filtered = {db_column_mapping[key]: row[key] for key in row.keys() if key in db_column_mapping.keys()}
                    cursor_destination = conn.cursor()
                    cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;", (ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                    count = cursor_destination.fetchone()[0]
                    if count == 0:
                        print(f"{ligne_filtered['NAME']} {ligne_filtered['SURNAME']} doesn't exist")
                    else:
                        cursor_destination.execute("UPDATE General SET GRADE_LV2=?, EXTRA_TIME=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["GRADE_LV2"], extra_time, ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                    conn.commit()
                    
            else:
                print(f"Format de fichier non pris en charge: {file}")
        
        else :
            db_column_mapping = {
                'Nom': 'NAME',
                'Prénom': 'SURNAME',
                'Mail': 'EMAIL',
                'Note/10' : 'GRADE_LV1'
            }
            if "_TT" in os.path.splitext(os.path.basename(file))[0]: 
                language = os.path.splitext(os.path.basename(file))[0]
                language = language[:-3]
                extra_time = True
                if file.endswith('.csv'):
                    csv_file_path = os.path.join(depot_note_folder, file)
                    with open(csv_file_path, 'r', encoding='utf-8-sig') as csvfile:
                        csv_reader = pd.read_csv(csv_file_path, encoding='utf-8-sig')
                        csv_reader['Note/10'] = csv_reader['Note/10'].replace('', np.nan)
                        csv_reader['Note/10'] = csv_reader['Note/10'].astype(float)
                        for ligne in csv_reader:
                            ligne_filtered = {key: value for key, value in ligne.items() if key in db_column_mapping.keys()}
                            cursor_destination = conn.cursor()
                            cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;", (ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))                    
                            count = cursor_destination.fetchone()[0]
                            if count == 0:
                                print(f"{ligne_filtered['NAME']} {ligne_filtered['SURNAME']} doesn't exist")
                            else:
                                cursor_destination.execute("UPDATE General SET GRADE_LV1=?, LV1=?, EXTRA_TIME=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["GRADE_LV1"], language, extra_time, ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                            conn.commit()

                elif file.endswith('.json'):
                    json_file_path = os.path.join(depot_note_folder, file)
                    with open(json_file_path, 'r', encoding='utf-8-sig') as jsonfile:
                        data = json.load(jsonfile)
                        df = pd.DataFrame(data)
                        df['Note/10'] = df['Note/10'].replace('', np.nan).astype(float)
                        for index, row in df.iterrows():
                            ligne_filtered = {db_column_mapping[key]: row[key] for key in row.keys() if key in db_column_mapping.keys()}
                            cursor_destination = conn.cursor()
                            cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;",(ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                            count = cursor_destination.fetchone()[0]
                            if count == 0:
                                print(f"{ligne_filtered['NAME']} {ligne_filtered['SURNAME']} doesn't exist")
                            else:
                                cursor_destination.execute("UPDATE General SET GRADE_LV1=?, LV1=?, EXTRA_TIME=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["GRADE_LV1"], language, extra_time, ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                            conn.commit()

                elif file.endswith('.xlsx'):
                    xlsx_file_path = os.path.join(depot_note_folder, file)
                    df = pd.read_excel(xlsx_file_path)
                    df['Note/10'] = df['Note/10'].replace('', np.nan).astype(float)
                    for index, row in df.iterrows():
                        ligne_filtered = {db_column_mapping[key]: row[key] for key in row.keys() if key in db_column_mapping.keys()}
                        cursor_destination = conn.cursor()
                        cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;", (ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                        count = cursor_destination.fetchone()[0]
                        if count == 0:
                            print(f"{ligne_filtered['NAME']} {ligne_filtered['SURNAME']} doesn't exist")
                        else:
                                cursor_destination.execute("UPDATE General SET GRADE_LV1=?, LV1=?, EXTRA_TIME=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["GRADE_LV1"], language, extra_time, ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                        conn.commit()
                else:
                    print(f"Format de fichier non pris en charge: {file}")
                
            else :
                extra_time = False
                if file.endswith('.csv'):
                    csv_file_path = os.path.join(depot_note_folder, file)
                    with open(csv_file_path, 'r', encoding='utf-8-sig') as csvfile:
                        csv_reader = pd.read_csv(csv_file_path, encoding='utf-8-sig')
                        csv_reader['Note/10'] = csv_reader['Note/10'].replace('', np.nan)
                        csv_reader['Note/10'] = csv_reader['Note/10'].astype(float)
                        for ligne in csv_reader:
                            ligne_filtered = {key: value for key, value in ligne.items() if key in db_column_mapping.keys()}
                            cursor_destination = conn.cursor()
                            cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;", (ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))                    
                            count = cursor_destination.fetchone()[0]
                            if count == 0:
                                print(f"{ligne_filtered['NAME']} {ligne_filtered['SURNAME']} doesn't exist")
                            else:
                                cursor_destination.execute("UPDATE General SET GRADE_LV1=?, LV1=?, EXTRA_TIME=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["GRADE_LV1"], os.path.splitext(os.path.basename(file))[0], extra_time, ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                            conn.commit()
                            
                elif file.endswith('.json'):
                    json_file_path = os.path.join(depot_note_folder, file)
                    with open(json_file_path, 'r', encoding='utf-8-sig') as jsonfile:
                        data = json.load(jsonfile)
                        df = pd.DataFrame(data)
                        df['Note/10'] = df['Note/10'].replace('', np.nan).astype(float)
                        for index, row in df.iterrows():
                            ligne_filtered = {db_column_mapping[key]: row[key] for key in row.keys() if key in db_column_mapping.keys()}
                            cursor_destination = conn.cursor()
                            cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;",(ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                            count = cursor_destination.fetchone()[0]
                            if count == 0:
                                print(f"{ligne_filtered['NAME']} {ligne_filtered['SURNAME']} doesn't exist")
                            else:
                                cursor_destination.execute("UPDATE General SET GRADE_LV1=?, LV1=?, EXTRA_TIME=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["GRADE_LV1"], os.path.splitext(os.path.basename(file))[0], extra_time, ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                            conn.commit()

                elif file.endswith('.xlsx'):
                    xlsx_file_path = os.path.join(depot_note_folder, file)
                    df = pd.read_excel(xlsx_file_path)
                    df['Note/10'] = df['Note/10'].replace('', np.nan).astype(float)
                    for index, row in df.iterrows():
                        ligne_filtered = {db_column_mapping[key]: row[key] for key in row.keys() if key in db_column_mapping.keys()}
                        cursor_destination = conn.cursor()
                        cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;", (ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                        count = cursor_destination.fetchone()[0]
                        if count == 0:
                            print(f"{ligne_filtered['NAME']} {ligne_filtered['SURNAME']} doesn't exist")
                        else:
                                cursor_destination.execute("UPDATE General SET GRADE_LV1=?, LV1=?, EXTRA_TIME=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["GRADE_LV1"], os.path.splitext(os.path.basename(file))[0], extra_time, ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                        conn.commit()
                else:
                    print(f"Format de fichier non pris en charge: {file}")

    conn.close()
    return




depot_note_folder ='./data/depot_notes'
db_path ='./data/SQL/data.sqlite3' 

load_survey(depot_note_folder,db_path)

### recuperer list classe & langue

In [61]:
import sqlite3

Data = 'data/SQL/data.sqlite3'

def find_list_CLASS(Data):
    conn = sqlite3.connect(Data)
    cursor = conn.cursor()
    cursor.execute("SELECT DISTINCT(CLASS) FROM General;")
    list_Class = cursor.fetchall()
    list_Class = [row[0] for row in list_Class]
    conn.close()
    return list_Class

find_list_CLASS(Data)

['1A']

In [174]:
import sqlite3

Data = 'data/SQL/data.sqlite3'

def find_list_LV2(Data):
    conn = sqlite3.connect(Data)
    cursor = conn.cursor()
    cursor.execute("SELECT DISTINCT(LV2) FROM General;")
    list_lv2 = cursor.fetchall()
    list_lv2 = [row[0] for row in list_lv2]
    conn.close()
    return list_lv2

def find_list_LV1(Data):
    conn = sqlite3.connect(Data)
    cursor = conn.cursor()
    cursor.execute("SELECT DISTINCT(LV1) FROM General;")
    list_lv2 = cursor.fetchall()
    list_lv2 = [row[0] for row in list_lv2]
    conn.close()
    return list_lv2

find_list_LV1(Data)

['Anglais']

### create groupe LV2

In [169]:
import sqlite3
import numpy as np

Data = 'data/SQL/data.sqlite3'
nb_by_class = 16
nb_forecast_by_class = {
        '1A': 30,
    }


def nomber_class(nomber_student, nomber_by_class):
    nomber_class = 0
    # à expliquer 
    if (nomber_student % nomber_by_class) > ((nomber_student //nomber_by_class)*(2/3)) :
        nomber_class = nomber_student//nomber_by_class + 1
    else :
        nomber_class = nomber_student//nomber_by_class
    return nomber_class

def make_groups(list_student, number_class):    
    total_students = len(list_student)
    students_per_group = total_students // number_class
    extra_students = total_students % number_class
    groups = []
    start_idx = 0
    for i in range(number_class):
        end_idx = start_idx + students_per_group
        if i < extra_students//2 :
            end_idx = end_idx
        elif i <  extra_students +  extra_students//2:  
            end_idx += 1
        groups.append(list_student[start_idx:end_idx])
        start_idx = end_idx
    return groups




def made_LV2_groupe(Data,nb_forecast_by_class, nb_by_class):
    for name_class in find_list_CLASS(Data):
        nb_forecast = nb_forecast_by_class[name_class]
        if nb_forecast== 0:
            for name_lv2 in find_list_LV2(Data):
                if "-débutant (jamais étudié)" in name_lv2 :
                    conn = sqlite3.connect(Data)
                    cursor = conn.cursor()
                    cursor.execute("SELECT EMAIL FROM General WHERE CLASS='" + name_class + "'AND LV2='" + name_lv2 + "';")
                    group = cursor.fetchall()
                    group = [pos[0] for pos in group]
                    nb_class = nomber_class(len(group),nb_by_class)
                    groups = make_groups(group, nb_class)
                    i = 1
                    for group in groups:
                        name_group = 'G' +str(i)        #donner un nom plus explicite
                        conn = sqlite3.connect(Data)
                        cursor = conn.cursor()
                        for student in group:
                            cursor.execute("UPDATE General SET GROUP_LV2=? WHERE EMAIL=?;", (name_group, student))
                            conn.commit()  
                            cursor.fetchall()  
                        i+=1
                else : 
                    conn = sqlite3.connect(Data)
                    cursor = conn.cursor()
                    cursor.execute("SELECT EMAIL,GRADE_LV2 FROM General WHERE CLASS='" + name_class + "'AND LV2='" + name_lv2 + "' ORDER BY GRADE_LV2 DESC;")
                    group = cursor.fetchall()
                    nb_class = nomber_class(len(group),nb_by_class)
                    groups = make_groups(group, nb_class)
                    i = 1
                    for group in groups:
                        name_group = 'G' +str(i)        #donner un nom plus explicite
                        conn = sqlite3.connect(Data)
                        cursor = conn.cursor()
                        for student in group:
                            cursor.execute("UPDATE General SET GROUP_LV2=? WHERE EMAIL=?;", (name_group, student[0]))
                            conn.commit()  
                            cursor.fetchall()  
                        i+=1  
            conn.close()                                                           
        else :
            conn = sqlite3.connect(Data)
            for name_lv2 in find_list_LV2(Data):
                cursor = conn.cursor()
                cursor.execute("SELECT count(*) FROM General WHERE CLASS='" + name_class + "';")
                total = cursor.fetchone()[0]  
                cursor.execute("SELECT count(*) FROM General WHERE CLASS='" + name_class + "' AND LV2='" + name_lv2 + "';")
                group = cursor.fetchone()[0]  
                if total != 0:
                    ratio = group / total * nb_forecast_by_class[name_class]
                    rounded_ratio = round(ratio)  
                if "-débutant (jamais étudié)" in name_lv2 :
                    cursor.execute("SELECT EMAIL FROM General WHERE CLASS='" + name_class + "'AND LV2='" + name_lv2 + "';")
                    group = cursor.fetchall()
                    group = [pos[0] for pos in group]
                    for i in range(rounded_ratio):
                        group.append('forecast')
                    nb_class = nomber_class(len(group),nb_by_class)
                    groups = make_groups(group, nb_class)
                    i = 1
                    for group in groups:
                        name_group = 'G' +str(i)        #donner un nom plus explicite
                        conn = sqlite3.connect(Data)
                        cursor = conn.cursor()
                        for student in group:
                            if student !='forecast':
                                cursor.execute("UPDATE General SET GROUP_LV2=? WHERE EMAIL=?;", (name_group, student))
                                conn.commit()  
                                cursor.fetchall()  
                        i+=1
                else : 
                    cursor.execute("SELECT EMAIL,GRADE_LV2 FROM General WHERE CLASS='" + name_class + "'AND LV2='" + name_lv2 + "' ORDER BY GRADE_LV2 DESC;")
                    group = cursor.fetchall()
                    numeric_values = [valeur for _, valeur in group if isinstance(valeur, (int, float))]
                    mu = np.mean(numeric_values)
                    sigma = np.var(numeric_values)**0.5
                    for i in range(rounded_ratio):
                        valeur_gaussienne = str(np.random.normal(mu, sigma))
                        group.append(('forecast',float(valeur_gaussienne)))
                    group = sorted(group, key=lambda x: x[1])
                    nb_class = nomber_class(len(group),nb_by_class)
                    groups = make_groups(group, nb_class)
                    i = 1
                    for group in groups:
                        name_group = 'G' +str(i)        #donner un nom plus explicite
                        cursor = conn.cursor()
                        for student in group:
                            if student !='forecast':
                                cursor.execute("UPDATE General SET GROUP_LV2=? WHERE EMAIL=?;", (name_group, student[0]))
                                conn.commit()  
                                cursor.fetchall()  
                        i+=1
            conn.close()
    return



made_LV2_groupe(Data,nb_forecast_by_class, nb_by_class)

### group LV1

In [175]:
import sqlite3
import numpy as np

Data = 'data/SQL/data.sqlite3'
nb_by_class = 16
nb_forecast_by_class = {
        '1A': 0,
    }


def nomber_class(nomber_student, nomber_by_class):
    nomber_class = 0
    # à expliquer 
    if (nomber_student % nomber_by_class) > ((nomber_student //nomber_by_class)*(2/3)) :
        nomber_class = nomber_student//nomber_by_class + 1
    else :
        nomber_class = nomber_student//nomber_by_class
    return nomber_class

def make_groups(list_student, number_class):    
    total_students = len(list_student)
    students_per_group = total_students // number_class
    extra_students = total_students % number_class
    groups = []
    start_idx = 0
    for i in range(number_class):
        end_idx = start_idx + students_per_group
        if i < extra_students//2 :
            end_idx = end_idx
        elif i <  extra_students +  extra_students//2:  
            end_idx += 1
        groups.append(list_student[start_idx:end_idx])
        start_idx = end_idx
    return groups




def made_LV1_groupe(Data,nb_forecast_by_class, nb_by_class):
    for name_class in find_list_CLASS(Data):
        nb_forecast = nb_forecast_by_class[name_class]
        if nb_forecast== 0:
            conn = sqlite3.connect(Data)            
            for name_lv1 in find_list_LV1(Data):
                cursor = conn.cursor()
                cursor.execute("SELECT EMAIL,GRADE_LV1 FROM General WHERE CLASS='" + name_class + "'AND LV1='" + name_lv1 + "' ORDER BY GRADE_LV1 DESC;")
                group = cursor.fetchall()
                nb_class = nomber_class(len(group),nb_by_class)
                print(nb_class)
                groups = make_groups(group, nb_class)
                i = 1
                for group in groups:
                    name_group = 'G' +str(i)        #donner un nom plus explicite
                    conn = sqlite3.connect(Data)
                    cursor = conn.cursor()
                    for student in group:
                        cursor.execute("UPDATE General SET GROUP_LV1=? WHERE EMAIL=?;", (name_group, student[0]))
                        conn.commit()  
                        cursor.fetchall()  
                    i+=1  
            conn.close()                                                           
        else :
            conn = sqlite3.connect(Data)
            for name_lv1 in find_list_LV1(Data):
                cursor = conn.cursor()
                cursor.execute("SELECT count(*) FROM General WHERE CLASS='" + name_class + "';")
                total = cursor.fetchone()[0]  
                cursor.execute("SELECT count(*) FROM General WHERE CLASS='" + name_class + "' AND LV1='" + name_lv1 + "';")
                group = cursor.fetchone()[0]  
                if total != 0:
                    ratio = group / total * nb_forecast_by_class[name_class]
                    rounded_ratio = round(ratio)  
                cursor.execute("SELECT EMAIL,GRADE_LV1 FROM General WHERE CLASS='" + name_class + "'AND LV1='" + name_lv1 + "' ORDER BY GRADE_LV1 DESC;")
                group = cursor.fetchall()
                numeric_values = [valeur for _, valeur in group if isinstance(valeur, (int, float))]
                mu = np.mean(numeric_values)
                sigma = np.var(numeric_values)**0.5
                for i in range(rounded_ratio):
                    valeur_gaussienne = str(np.random.normal(mu, sigma))
                    group.append(('forecast',float(valeur_gaussienne)))
                group = sorted(group, key=lambda x: x[1])
                nb_class = nomber_class(len(group),nb_by_class)
                groups = make_groups(group, nb_class)
                i = 1
                for group in groups:
                    name_group = 'G' +str(i)        #donner un nom plus explicite
                    cursor = conn.cursor()
                    for student in group:
                        if student !='forecast':
                            cursor.execute("UPDATE General SET GROUP_LV1=? WHERE EMAIL=?;", (name_group, student[0]))
                            conn.commit()  
                            cursor.fetchall()  
                    i+=1
            conn.close()
    return



made_LV1_groupe(Data,nb_forecast_by_class, nb_by_class)

12
